# Classification of Arrhythmia by Using Deep Learning with 2-D ECG Spectral Image Representation

In [1]:
# import the libraries

In [2]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense

In [3]:
from tensorflow.keras.layers import Convolution2D
from tensorflow.keras.layers import MaxPooling2D
from tensorflow.keras.layers import Flatten

In [4]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

In [5]:
pwd

'/home/wsuser/work'

In [6]:
ls

In [7]:

import os, types
import pandas as pd
from botocore.client import Config
import ibm_boto3

def __iter__(self): return 0

# @hidden_cell
# The following code accesses a file in your IBM Cloud Object Storage. It includes your credentials.
# You might want to remove those credentials before you share the notebook.
cos_client = ibm_boto3.client(service_name='s3',
    ibm_api_key_id='0fhc2oHPirg2Dm97_F7SaezVuuQTeNyclTy34qo1nh-r',
    ibm_auth_endpoint="https://iam.cloud.ibm.com/oidc/token",
    config=Config(signature_version='oauth'),
    endpoint_url='https://s3.private.us.cloud-object-storage.appdomain.cloud')

bucket = 'classificationofarrhythmia-donotdelete-pr-i20ljpgtr2g4v8'
object_key = 'Classification of Arrhythmia by Using Deep Learning with 2-D ECG Spectral Image Representation.zip'

streaming_body_2 = cos_client.get_object(Bucket=bucket, Key=object_key)['Body']

# Your data file was loaded into a botocore.response.StreamingBody object.
# Please read the documentation of ibm_boto3 and pandas to learn more about the possibilities to load the data.
# ibm_boto3 documentation: https://ibm.github.io/ibm-cos-sdk-python/
# pandas documentation: http://pandas.pydata.org/


In [8]:
from io import BytesIO
import zipfile
unzip=zipfile.ZipFile(BytesIO(streaming_body_2.read()),'r')
file_path=unzip.namelist()
for path in file_path:
    unzip.extract(path)

In [9]:
ls

data/


In [10]:
train_datagen = ImageDataGenerator(rescale=1./255,shear_range=0.2,zoom_range=0.2,horizontal_flip=True,vertical_flip=True)


In [11]:
test_datagen = ImageDataGenerator(rescale=1./255)

In [12]:
#load my images data

In [13]:
x_train = train_datagen.flow_from_directory(r"/home/wsuser/work/data/train",target_size=(64,64),batch_size=32,class_mode="categorical")

Found 15341 images belonging to 6 classes.


In [14]:
x_test = test_datagen.flow_from_directory(r"/home/wsuser/work/data/test",target_size=(64,64),batch_size=32,class_mode="categorical")

Found 6825 images belonging to 6 classes.


In [15]:
x_train.class_indices

{'Left Bundle Branch Block': 0,
 'Normal': 1,
 'Premature Atrial Contraction': 2,
 'Premature Ventricular Contractions': 3,
 'Right Bundle Branch Block': 4,
 'Ventricular Fibrillation': 5}

In [16]:
#initialize the model

In [17]:
model = Sequential()

In [18]:
#add convolution layer
model.add(Convolution2D(32,(3,3),input_shape=(64,64,3),activation='relu'))

In [19]:
#add maxpooling layer 
model.add(MaxPooling2D(pool_size=(2,2)))

In [20]:
#flatten layer =>input layer to your ANN
model.add(Flatten())

In [21]:
#hidden layers

In [22]:
model.add(Dense(units=300,kernel_initializer="random_uniform",activation="relu"))

In [23]:
model.add(Dense(units=500,kernel_initializer="random_uniform",activation="relu"))

In [24]:
model.add(Dense(units=600,kernel_initializer="random_uniform",activation="relu"))

In [25]:
model.add(Dense(units=500,kernel_initializer="random_uniform",activation="relu"))

In [26]:
model.add(Dense(units=300,kernel_initializer="random_uniform",activation="relu"))

In [27]:
model.add(Dense(units=200,kernel_initializer="random_uniform",activation="relu"))

In [28]:
#output layer

In [29]:
model.add(Dense(units=6,kernel_initializer="random_uniform",activation="softmax"))

In [30]:
#compile the model

In [31]:
model.compile(loss="categorical_crossentropy",optimizer="adam",metrics=["accuracy"])

In [32]:
#train the model

In [33]:
model.fit_generator(x_train,steps_per_epoch=480,epochs=25,validation_data=x_test,validation_steps=106)

/tmp/wsuser/ipykernel_164/4167339887.py:1: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  model.fit_generator(x_train,steps_per_epoch=480,epochs=25,validation_data=x_test,validation_steps=106)


Epoch 1/25
480/480 [==============================] - 102s 212ms/step - loss: 1.3195 - accuracy: 0.5086 - val_loss: 1.4677 - val_accuracy: 0.3682
Epoch 2/25
480/480 [==============================] - 101s 209ms/step - loss: 0.8495 - accuracy: 0.6518 - val_loss: 1.2872 - val_accuracy: 0.5333
Epoch 3/25
480/480 [==============================] - 100s 209ms/step - loss: 0.4852 - accuracy: 0.8281 - val_loss: 0.8334 - val_accuracy: 0.7801
Epoch 4/25
480/480 [==============================] - 100s 209ms/step - loss: 0.3052 - accuracy: 0.9014 - val_loss: 0.7171 - val_accuracy: 0.7854
Epoch 5/25
480/480 [==============================] - 100s 208ms/step - loss: 0.2211 - accuracy: 0.9342 - val_loss: 0.6527 - val_accuracy: 0.8210
Epoch 6/25
480/480 [==============================] - 100s 208ms/step - loss: 0.1718 - accuracy: 0.9480 - val_loss: 0.5576 - val_accuracy: 0.8535
Epoch 7/25
480/480 [==============================] - 100s 209ms/step - loss: 0.1525 - accuracy: 0.9521 - val_loss: 0.5418 -

## save the model

In [53]:
#save the model
model.save("model.h5")

In [37]:
##IBM Deployment

In [38]:
!pip install watson-machine-learning-client

In [42]:
from ibm_watson_machine_learning import APIClient
wml_credentials={
    "url":"https://us-south.ml.cloud.ibm.com",
    "apikey":"t3hXSd_XTFXIHjD91HYv15_efbGfTGo-d_miKapL3J7r"
}
client=APIClient(wml_credentials)

In [44]:
client

In [45]:
def guid_space_name(client,Arrhythmia_deploy):
    space=client.spaces.get_details()
    return(next(item for item in space['resources'] if item['entity']['name']==Arrhythmia_deploy)['metadata']['id'])

In [47]:
space_uid=guid_space_name(client,'Arrhythmia_deploy')
print("Space UID"+space_uid)

Space UIDc3487e94-25de-4ffd-8a41-9616c7febe9a


In [48]:
client.set.default_space(space_uid)

'SUCCESS'

In [49]:
client.software_specifications.list(200)

-------------------------------  ------------------------------------  ----
NAME                             ASSET_ID                              TYPE
default_py3.6                    0062b8c9-8b7d-44a0-a9b9-46c416adcbd9  base
kernel-spark3.2-scala2.12        020d69ce-7ac1-5e68-ac1a-31189867356a  base
pytorch-onnx_1.3-py3.7-edt       069ea134-3346-5748-b513-49120e15d288  base
scikit-learn_0.20-py3.6          09c5a1d0-9c1e-4473-a344-eb7b665ff687  base
spark-mllib_3.0-scala_2.12       09f4cff0-90a7-5899-b9ed-1ef348aebdee  base
pytorch-onnx_rt22.1-py3.9        0b848dd4-e681-5599-be41-b5f6fccc6471  base
ai-function_0.1-py3.6            0cdb0f1e-5376-4f4d-92dd-da3b69aa9bda  base
shiny-r3.6                       0e6e79df-875e-4f24-8ae9-62dcc2148306  base
tensorflow_2.4-py3.7-horovod     1092590a-307d-563d-9b62-4eb7d64b3f22  base
pytorch_1.1-py3.6                10ac12d6-6b30-4ccd-8392-3e922c096a92  base
tensorflow_1.15-py3.6-ddl        111e41b3-de2d-5422-a4d6-bf776828c4b7  base
autoai-kb_rt

In [50]:
software_space_uid=client.software_specifications.get_uid_by_name('tensorflow_rt22.1-py3.9')

In [51]:
software_space_uid

'acd9c798-6974-5d2f-a657-ce06e986df4d'

In [56]:
ls

classification-arrhythmia-model.tgz  data/  model.h5


In [55]:
!tar -zcvf classification-arrhythmia-model.tgz model.h5

model.h5


In [58]:
model_details=client.repository.store_model(model='classification-arrhythmia-model.tgz',meta_props={
    client.repository.ModelMetaNames.NAME:"Arrhythmia model Building",
    client.repository.ModelMetaNames.TYPE:'tensorflow_2.7',
    client.repository.ModelMetaNames.SOFTWARE_SPEC_UID:software_space_uid
})

In [59]:
model_id=client.repository.get_model_id(model_details)

In [60]:
model_id

'25e87365-5894-4093-a14c-ecb232a7d90b'

In [61]:
client.repository.download(model_id,'classification-arrhythmia-model.tgz.gp')

Successfully saved model content to file: 'classification-arrhythmia-model.tgz.gp'


'/home/wsuser/work/classification-arrhythmia-model.tgz.gp'